In [ ]:
from Levenshtein import ratio, jaro, distance
from collections import Counter

import pandas as pd
import requests
import subprocess
import gzip
import json
import math
import os
import re

def set_default_path(directory: str | None = 'Event-Aggregator') -> str:
    """Changes the working directory to """
    while not os.getcwd().endswith(directory):
        os.chdir('..')
        if len(os.getcwd()) == 0:
            raise FileExistsError
    return os.getcwd()

def query(url, data, mode):
    match mode:
        case 'POST':
            item = requests.post(url,json=data)
            item = json.loads(item.content.decode())
        case 'GET':
            item = requests.get(url)
            item = json.loads(item.content.decode())
    return item

default_path = os.path.join(set_default_path() ,'Production_Environment')
server_path  = os.path.join(default_path,'Servers') 
collectors_path = os.path.join(server_path,'Collectors')
events_path  = os.path.join(collectors_path,'Events')
data_path    = os.path.join(events_path,'Data')
File_path    = os.path.join(data_path, 'Final_Events.csv')

In [28]:
df = pd.read_csv('Final_Events.csv')

In [54]:
n_rows, n_columns = df.shape
n_rows,n_columns

(489, 9)

In [57]:
list(df.columns)

['Title',
 'Desc',
 'Start Date',
 'When',
 'Main Page',
 'Address1',
 'Address2',
 'thumb',
 'image']

In [76]:
results  = []
for idx in range(n_rows):
    row = df.iloc[idx].fillna('')
    item = {
        'TITLE': row['Title'],
        'DATE' : row['Start Date'],
        'MAINPAGE' : row['Main Page'],
        'ADDRESS1' : row['Address1'],
        'ADDRESS2' : row['Address2'],
        'WEHN' : row['When'],
        'THUMB': row['thumb'],
        'IMAGE' : row['image'],
        'DESC' : row['Desc'],
        'LongDesc': ''
    }
    results.append(item)


In [104]:
url  = "http://127.0.0.1:5000/sample"
    
query(url = url, data = None, mode = 'GET')

[[15,
  'Marvel Rivals In-Person Stream Viewing Esports',
  'Mar 2',
  'https://www.eventbrite.com/e/marvel-rivals-in-person-stream-viewing-esports-tickets-1260447184459',
  'Santa Anita Park, 285 W Huntington Dr',
  'Arcadia, CA',
  'Sun, Mar 2, 11 AM – 1 PM PST',
  'https://www.google.com/maps/vt/data=WgsdpaHj5ciyKNJ2yn_8P0wSjO4gV3hsgS4sfCx47XxFN0O94FAKMZeYrF4zlicCaW5H1nGIt_QM-72X_3Xv7EBbxZF4JZvuHZcX6vio9zjm6Mnd8Iw',
  '',
  'We are hosting an in-person meetup to watch a popular streamer together! Come have a great time! We are excited to watch esports together!',
  '']]

In [102]:
url  = "http://127.0.0.1:5000/save_events"
    
select = {
    'COL': 'DESC'
}
query(url = url, data = results, mode = 'POST')

{'Message': 'ok'}